In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd
from utils.path import WildcardPath
import os
from utils.enums import (
    AlignerEnum,
    FeatureEnum as FE,
    FeatureDirectionEnum as FDE,
    FeatureTypeEnum as FTE,
)
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import alignment.ensemble
import alignment.dataset
import utils.plots

In [2]:
DATA_DIR = WildcardPath("/hits/fast/cme/bodynems/data/")
RESULT_DIR = WildcardPath("/hits/fast/cme/bodynems/MSA_difficulty/results")

# BENCHMARKS = ["balibase3/RV11", "balibase3/RV12"]
benchmark = "treebase_large"
INPUT_DIR = DATA_DIR / "input" / "{benchmark}"
OUTPUT_DIR = DATA_DIR / "output" / "{benchmark}"

DATASET_DIR = OUTPUT_DIR / "{dataset}"


aligned_FEAT_FILE = "aligned_features.parquet"
TOOLS = [aligner for aligner in AlignerEnum]

In [3]:
# dist_mats contains one distance matrix per dataset.
# The matrices have shape (n_replicates, n_replicates).
dist_mats = {}
reference_features = []
for dataset in OUTPUT_DIR.format(benchmark=benchmark).listdir(dirs_only=True):
    dist_mat_dir = (
        DATASET_DIR.format(benchmark=benchmark, dataset=dataset) / "distances"
    )
    for file in dist_mat_dir.listdir():
        print(file)
        metric = file.split(".")[0]
        dist_mats[(benchmark, dataset, metric)] = pd.read_parquet(dist_mat_dir / file)
    break

#     benchmark_ref_df = pd.read_parquet(
#         OUTPUT_DIR.format(benchmark=benchmark) / "reference_features.parquet"
#     )
#     benchmark_ref_df["benchmark"] = benchmark
#     reference_features.append(benchmark_ref_df)
# reference_features = pd.concat(reference_features, axis=0, ignore_index=True)

displacement_dist_d0.parquet
displacement_dist_d8.parquet
displacement_dist_d32.parquet
homology_pos_dist.parquet
gap_delta_dist.parquet
displacement_dist_d2.parquet
perc_hash_hamming_dist_256bit.parquet
displacement_dist_d16.parquet
perc_hash_hamming_dist_16bit.parquet
homology_seq_dist.parquet
perc_hash_hamming_dist_64bit.parquet
displacement_dist_d4.parquet
displacement_dist_d1.parquet
homology_raw_dist.parquet
displacement_dist.parquet
symmetrized_sp_dist.parquet


In [17]:
for dist_mat in dist_mats.values():
    print(dist_mat.index)
    break
print(len(dist_mats))

Index(['Mafft_FFT-NS-2', 'Mafft_FFT-NS-2', 'Mafft_FFT-NS-2', 'Mafft_FFT-NS-2',
       'Mafft_FFT-NS-2', 'Mafft_FFT-NS-2', 'Mafft_FFT-NS-2', 'Mafft_FFT-NS-2',
       'Mafft_L-INS-i', 'Mafft_L-INS-i', 'Mafft_L-INS-i', 'Mafft_L-INS-i',
       'Mafft_L-INS-i', 'Mafft_L-INS-i', 'Mafft_L-INS-i', 'Mafft_L-INS-i',
       'Mafft_G-INS-i', 'Mafft_G-INS-i', 'Mafft_G-INS-i', 'Mafft_G-INS-i',
       'Mafft_G-INS-i', 'Mafft_G-INS-i', 'Mafft_G-INS-i', 'Mafft_G-INS-i',
       'ClustalO', 'ClustalO', 'ClustalO', 'ClustalO', 'ClustalO', 'ClustalO',
       'ClustalO', 'ClustalO', 'Muscle5', 'Muscle5', 'Muscle5', 'Muscle5',
       'Muscle5', 'Muscle5', 'Muscle5', 'Muscle5', 'Muscle3', 'Muscle3',
       'Muscle3', 'Muscle3', 'Muscle3', 'Muscle3', 'Muscle3', 'Muscle3'],
      dtype='object')
16


In [10]:
# score_mats contains a list of ref-rep distances for each dataset.
# The lists have length n_replicates.
ref_rep_dists_map = {}

for benchmark, dataset, metric in dist_mats.keys():
    ref_rep_dists = (
        reference_features.query(
            f"(benchmark == '{benchmark}') & (dataset == '{dataset}')"
        )
        .loc[:, "median_" + FE.HOMOLOGY_POS_DIST]
        .reset_index(drop=True)
    )
    ref_rep_dists_map[(benchmark, dataset)] = ref_rep_dists

In [4]:
from sklearn.decomposition import PCA
import seaborn as sn

pca = PCA(n_components=2)
for i, (benchmark, dataset, metric) in enumerate(dist_mats):
    # if metric == FE.HOMOLOGY_POS_DIST:
    if metric == FE.HOMOLOGY_POS_DIST:
        print(metric)
        out_path = RESULT_DIR / benchmark / "plots" / "mds"
        os.makedirs(out_path, exist_ok=True)
        dist_mat = dist_mats[(benchmark, dataset, metric)]
        # dist_mat = dist_mat + np.random.random(dist_mat.shape) * 1e-4
        # sn.heatmap(dist_mat)
        # plt.show()
        color = None
        # color = ref_rep_dists_map[(benchmark, dataset)]
        fig = utils.plots.mds_tool_scatterplot(
            dist_mat,
            dataset,
            color,
            metric_name=metric,
            out_dir=out_path,
            out_type="svg",
            method="metric",
        )
        # fig.show()
        plt.close(fig)
    # plt.show()
    # if i > 10:
    #     break

homology_pos_dist
22475_2.phy [0.5  0.13 0.08 0.06 0.04]
